### 代码参考： 
https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

In [ ]:
# Author: Robert Guthrie

from torchtext.legacy import data,datasets
from torchtext.vocab import Vectors

import pandas as pd

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

import unicodedata
import re
import codecs

# from BiLSTM_CRF import BiLSTM_CRF



In [ ]:

DEVICE=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

SEED=2022
torch.manual_seed(SEED)

In [ ]:
def zero_digits(s):
    """
    Replace every digit in a string by a zero.
    """
    return re.sub('\d', '0', s)

def load_sentences(path, lower, zeros):
    """
    Load sentences. A line must contain at least a word and its tag.
    Sentences are separated by empty lines.
    """
    text=[]
    label=[]
    text_list=[]
    label_list=[]
    for line in codecs.open(path, 'r', 'utf-8'):
        line = zero_digits(line.rstrip()) if zeros else line.rstrip()
        if not line:
            text_list.append(text)
            label_list.append(label)
            label=[]
            text=[]
        else:
            sample=line.split()
            word=sample[0]
            nertag=sample[-1]
            text.append(word)
            label.append(nertag)
    df_data=pd.DataFrame()
    df_data['text']=text_list
    df_data['label']=label_list

    return df_data

def laod_dataset(df,textfield,labelfield):

    examples=[]
    fields = [('text', textfield), ('label', labelfield)]
    for index,row in df.iterrows():
        examples.append(data.Example.fromlist([row['text'], row['label']], fields))
    dataset = data.Dataset(examples, fields)
    return dataset

root_path='../data/'
train_path=root_path+'eng.train'
valid_path=root_path+'eng.testa'
test_path=root_path+'eng.testb'

train_df=load_sentences(train_path,lower=True,zeros=True)
valid_df=load_sentences(valid_path,lower=True,zeros=True)
test_df=load_sentences(test_path,lower=True,zeros=True)

# TEXT=data.Field(sequential=True)
# LABEL = data.Field(sequential=False, use_vocab=False)

# train_data=laod_dataset(train_df,textfield=TEXT,labelfield=LABEL)
# valid_data=laod_dataset(valid_df,textfield=TEXT,labelfield=LABEL)
# test_data=laod_dataset(test_df,textfield=TEXT,labelfield=LABEL)

# EMBEDDING_DIM=100
# BATCH_SIZE=32
# MIN_FREQ=2

# TEXT.build_vocab(
#     train_data,valid_data,test_data,
#     min_freq=MIN_FREQ,
#     vectors="glove.6B.{}d".format(EMBEDDING_DIM),
#     unk_init=torch.Tensor.normal_
# )
# LABEL.build_vocab(train_data,valid_data,test_data)

# train_iterator,valid_iterator,test_iterator=data.BucketIterator.splits(
#     (train_data,valid_data,test_data),
#     batch_size=BATCH_SIZE,
#     device=DEVICE
# )


In [ ]:
START_TAG = "<START>"
STOP_TAG = "<STOP>"
HIDDEN_DIM = 100
EMBEDDING_DIM=100
EPOCH=5

# train_data look like this
# training_data = [(
#     "the wall street journal reported today that apple corporation made money".split(),
#     "B I I I O O O B I O O".split()
# ), (
#     "georgia tech is a university in georgia".split(),
#     "B I O O O O B".split()
# )]

word_to_ix = {}
tag_to_ix = {START_TAG: 0, STOP_TAG: 1}
for intex,row in train_df.iterrows():

    for word in row['text']:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

    for tag in row['label']:
         if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)



In [ ]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix) #k+2

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        print(init_alphas)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas
        print(forward_var)
        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                print('emit_score:',emit_score)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                print('trans_score',trans_score)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            print('forward_var:',forward_var)
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        # print(sentence.shape)
        feats = self._get_lstm_features(sentence)
        # print(feats.shape)
        forward_score = self._forward_alg(feats)
        # print(forward_score)
        gold_score = self._score_sentence(feats, tags)
        # print(gold_score)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

# Check predictions before training
# with torch.no_grad():
#     precheck_sent = prepare_sequence(train_df.iloc[0,0], word_to_ix)
#     precheck_tags = torch.tensor([tag_to_ix[t] for t in train_df.iloc[0,1]], dtype=torch.long)
#     print(model(precheck_sent))

precheck_sent = prepare_sequence(train_df.iloc[0,0], word_to_ix)
precheck_tags = torch.tensor([tag_to_ix[t] for t in train_df.iloc[0,1]], dtype=torch.long)
model.neg_log_likelihood(precheck_sent,precheck_tags)

Create model



In [ ]:
from tqdm import tqdm
import numpy as np
# Make sure prepare_sequence from earlier in the LSTM section is loaded
loss_mean=0
iter_num=500
for epoch in range(EPOCH):  # again, normally you would NOT do 300 epochs, it is toy data
    for index,row in tqdm(train_df.iterrows()):
        sentence=row['text']
        tags=row['label']
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)
        loss_mean+=loss
        if (index+1) % iter_num==0:
            print("%.2f" % (loss_mean/iter_num) )
            loss_mean=0
        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()


# We got it!

In [ ]:
# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    print(model(precheck_sent))

In [ ]:

# 此代码出问题了，如果用batch的话不知道怎么对文章进行一个embedding，还是使用原来的方案，就直接一个一个batch来。

# def zero_digits(s):
#     """
#     Replace every digit in a string by a zero.
#     """
#     return re.sub('\d', '0', s)

# def load_sentences(path, lower, zeros):
#     """
#     Load sentences. A line must contain at least a word and its tag.
#     Sentences are separated by empty lines.
#     """
#     text=[]
#     label=[]
#     text_list=[]
#     label_list=[]
#     for line in codecs.open(path, 'r', 'utf-8'):
#         line = zero_digits(line.rstrip()) if zeros else line.rstrip()
#         if not line:
#             text_list.append(text)
#             label_list.append(label)
#             label=[]
#             text=[]
#         else:
#             sample=line.split()
#             word=sample[0]
#             nertag=sample[-1]
#             text.append(word)
#             label.append(nertag)
#     df_data=pd.DataFrame()
#     df_data['text']=text_list
#     df_data['label']=label_list

#     return df_data

# def laod_dataset(df,textfield,labelfield):

#     examples=[]
#     fields = [('text', textfield), ('label', labelfield)]
#     for index,row in df.iterrows():
#         examples.append(data.Example.fromlist([row['text'], row['label']], fields))
#     dataset = data.Dataset(examples, fields)

#     return dataset

# root_path='../data/'
# train_path=root_path+'eng.train'
# valid_path=root_path+'eng.testa'
# test_path=root_path+'eng.testb'

# train_df=load_sentences(train_path,lower=True,zeros=True)
# valid_df=load_sentences(valid_path,lower=True,zeros=True)
# test_df=load_sentences(test_path,lower=True,zeros=True)

# TEXT=data.Field(sequential=True)
# LABEL = data.Field(sequential=False, use_vocab=False)

# train_data=laod_dataset(train_df,textfield=TEXT,labelfield=LABEL)
# valid_data=laod_dataset(valid_df,textfield=TEXT,labelfield=LABEL)
# test_data=laod_dataset(test_df,textfield=TEXT,labelfield=LABEL)

# EMBEDDING_DIM=100
# BATCH_SIZE=32
# MIN_FREQ=2

# TEXT.build_vocab(
#     train_data,valid_data,test_data,
#     min_freq=MIN_FREQ,
#     vectors="glove.6B.{}d".format(EMBEDDING_DIM),
#     unk_init=torch.Tensor.normal_
# )
# LABEL.build_vocab(train_data,valid_data,test_data)

# train_iterator,valid_iterator,test_iterator=data.BucketIterator.splits(
#     (train_data,valid_data,test_data),
#     batch_size=BATCH_SIZE,
#     device=DEVICE
# )